In [ ]:
# Third party packages
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
import matplotlib.pyplot as plt
from io import StringIO
from IPython.display import display
import plotly.express as px

# Adding path
from os.path import abspath
import sys
sys.path.insert(0, abspath('..'))

# 
from query_request import *
from query_def import *
import credentials
# from sqlalchemy import create_engine

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth=None


# [Federated Query Engine (DeTrusty)](https://github.com/SDM-TIB/DeTrusty)
**A federated query engine**

Step to run Detruty against the federation of `Knowledge Graphs`: 
* [Install and Run](https://github.com/SDM-TIB/DeTrusty)
* Create [Endpoints](../../configs/endpoints_sample.json) file and [RDF Molecule Template](../../configs/rdfmts_public.json)
* Run query using `Detrusty.run_query`

In [ ]:
# create rdfmts
from DeTrusty import run_query
from DeTrusty.Molecule.MTCreation import create_rdfmts
from DeTrusty.Molecule.MTManager import ConfigFile

# Creating endpoints dictionary
endpoints = dictfrmjson('../configs/endpoints_sample.json')
print(endpoints)

# Creating RDF molecule template (rdfmts) from endpoints
# log_rdfmts = create_rdfmts(endpoints, '../configs/rdfmts.json')

# if already rdfmts has been created 
config_private = ConfigFile('../configs/rdfmts_private.json')
config_public = ConfigFile('../configs/rdfmts_public.json')

## Endpoints Status

In [ ]:
# Demo query to all available enpoints

# demo query
query_test = """SELECT * WHERE{ ?Subject a ?Concept }LIMIT 1000"""


# creating request for cmemc
display("############### Cmemc Endpoint ###############")
cmemc_request = SPARQLRequest(client_url_imp, client_id_imp, client_secret_imp , 'oauth')
# executing query and printing response
cmemc_request.execute(query_test)
buf = StringIO(cmemc_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for skynet
display("############### Skynet Endpoint ###############")
skynet_request = SPARQLRequest(skynet_endpoint, skynet_user, skynet_pass, 'basic')
# executing query and printing response
skynet_request.execute(query_test)
buf = StringIO(skynet_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for worldbank
display("############### Worldbank Endpoint ###############")
worldbank_request = SPARQLRequest(worldbank_endpoint)
# executing query and printing response
worldbank_request.execute(query_test)
display(worldbank_request.response.status_code)
buf = StringIO(worldbank_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for LEI
display("############### LEI Endpoint ###############")
lei_request = SPARQLRequest(lei_endpoint)
# executing query and printing response
lei_request.execute(query_test)
buf = StringIO(lei_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for ICEWS
display("############### ICEWS Endpoint ###############")
icews_request = SPARQLRequest(icews_endpoint)
# executing query and printing response
icews_request.execute(query_test)
buf = StringIO(icews_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))


## Federated Query

In [ ]:
# creating request for Federated Query
display("############### Federated Query ###############")
fdq_request = SPARQLRequest(is_fdq=True)
fdq_request.execute(query_test, config_public, False)
fdq_request.save('../query_results/', 'query_test')

In [ ]:
# print(query_2_fdq_ex_1)

query = """PREFIX wb: <http://worldbank.org/>
PREFIX wbi: <http://worldbank.org/Indicator/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX p: <http://www.wikidata.org/prop/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?country_code ?country_name ?year (AVG(?year_exp) as ?year_avg_exp_wiki) (AVG(?year_exp_WB) as ?year_avg_exp_WB) 
WHERE {
    ?country a wb:Country .
    ?country dc:identifier ?country_code .
    ?country rdfs:label ?country_name.
    ?country owl:sameAs ?sameAsCountry .
    ?country wb:hasAnnualIndicatorEntry ?annualIndicator .
    
    ?annualIndicator wb:hasIndicator <http://worldbank.org/Indicator/SP.DYN.LE00.IN> .
    ?annualIndicator owl:hasValue ?year_exp_WB .
    ?annualIndicator time:year ?year .

    ?sameAsCountry p:P2250 ?itemLifeExpectancy .
    ?itemLifeExpectancy ps:P2250 ?year_exp .
    ?itemLifeExpectancy pq:P585 ?time .
    BIND(year(?time) AS ?year)
}
GROUP BY ?country_code ?country_name ?year
ORDER BY ?country_code
LIMIT 10
"""

fdq_request.execute(query, config_public, False)
fdq_request.save('../query_results/', 'query')

# Visualization

In [ ]:
df = pd.read_csv('../../query_results/query_2_fdq_ex_1.csv', encoding='utf-8')
df.rename(columns=lambda x: x.replace('.value', ''), inplace=True)
df['WikiData-WB(AvgLifeExpectancy)'] = np.abs(df['year_avg_exp_wiki']-df['year_avg_exp_WB'])
df.head(10)

In [ ]:
fig = px.choropleth(df[df['year_WB']==2016], locations="country_code",
                    color="WikiData-WB(AvgLifeExpectancy)", # lifeExp is a column of gapminder
                    hover_name="country_name", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Magenta)
fig.write_html('../../query_results/query_2_fdq_ex_1.html')
fig.show()